# Проверка на выбросы

In [13]:
import pandas as pd


df = pd.read_csv('Data/final_encoded.csv')
df.head()

,price,metro_time,info_total_area,okrug_ВАО,okrug_ЗАО,okrug_ЗелАО,okrug_НАО (Новомосковский),okrug_САО,okrug_СВАО,okrug_СЗАО,...,rent_living_conditions_можно с детьми,rent_living_conditions_можно с детьми и животными,rent_living_conditions_можно с животными,"rent_living_conditions_ни с детьми, ни с животными",year_built_interval_1890-1917,year_built_interval_1917-1945,year_built_interval_1945-1970,year_built_interval_1970-1990,year_built_interval_1991-2025,year_built_interval_не попал
0,470000,7,73,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,270000,13,68,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,30000,9,40,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,75000,13,60,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,29000,7,12,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0


# DFFITS

In [61]:
import numpy as np
import statsmodels.api as sm

# Создание модели
X = sm.add_constant(df.drop(columns='price')).values
y = df['price']
model = sm.OLS(y, X).fit()

# Ручной расчет DFFITS
residuals = y - model.predict(X)
h = np.diag(X @ np.linalg.inv(X.T @ X) @ X.T)  # Матрица шляпницы
p = X.shape[1]  # Количество параметров (включая константу)
n = len(y)

# Расчет стандартных ошибок
s_squared = np.sum(residuals**2) / (n - p)
s_i = np.sqrt(s_squared * (1 - h))

# Формула DFFITS
dffits = (residuals / s_i) * np.sqrt(h / (1 - h))

# Пороговое значение
dffits_threshold = 2 * np.sqrt(p / n)

# Фильтрация
non_outliers_mask = np.abs(dffits) <= dffits_threshold
data_filtered_no_outliers = df[non_outliers_mask].reset_index(drop=True)

print(f"Размер до фильтрации: {len(df)}")
print(f"Размер после фильтрации: {len(data_filtered_no_outliers)}")


Размер до фильтрации: 1390
Размер после фильтрации: 1169


/var/folders/02/gfr6gmcd60b3x__kfbgczszm0000gn/T/ipykernel_8234/2212241655.py:17: RuntimeWarning:

invalid value encountered in sqrt

/var/folders/02/gfr6gmcd60b3x__kfbgczszm0000gn/T/ipykernel_8234/2212241655.py:20: RuntimeWarning:

invalid value encountered in sqrt



In [60]:
import plotly.graph_objects as go


# Визуализация
fig = go.Figure()

# Точки DFFITS
fig.add_trace(go.Scatter(
    x=np.arange(len(dffits)),
    y=dffits,
    mode='markers',
    name='DFFITS',
    marker=dict(
        color=np.where(np.abs(dffits) > dffits_threshold, 'red', 'blue'),
        size=5,
        opacity=0.7
    ),
    hovertemplate="Наблюдение: %{x}<br>DFFITS: %{y:.3f}<extra></extra>"
))

# Пороговые линии
fig.add_hline(
    y=dffits_threshold,
    line_dash="dot",
    line_color="red",
    annotation_text=f"Верхний порог: {dffits_threshold:.2f}",
    annotation_position="top right"
)

fig.add_hline(
    y=-dffits_threshold,
    line_dash="dot",
    line_color="red",
    annotation_text=f"Нижний порог: {-dffits_threshold:.2f}",
    annotation_position="bottom right"
)

# Выделение выбросов
outliers = np.where(np.abs(dffits) > dffits_threshold)[0]
fig.add_trace(go.Scatter(
    x=outliers,
    y=dffits[outliers],
    mode='markers',
    marker=dict(
        color='red',
        size=8,
        symbol='x'
    ),
    name='Выбросы'
))

# Оформление
fig.update_layout(
    title='Анализ выбросов с помощью DFFITS',
    xaxis_title='Номер наблюдения',
    yaxis_title='DFFITS',
    hovermode='closest',
    showlegend=True,
    template='plotly_white',
    height=600,
    yaxis_range=[-4, 4],
)

fig.show()